In [95]:
import os
import sys
module_path = os.path.abspath(os.path.join('..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [96]:
import functools
%run ../gw/gw_wrapped_compressed.ipynb
%run ../compressed_class_list/compressed_class_list.ipynb
%run chiang_ogw.ipynb

IndexError: list index out of range

IndexError: list index out of range

-175/116
w:29/64
b: 2, A: [3], chiang ogw: 1
b: 3, A: [2, 3], chiang ogw: -3/8
b: 4, A: [2, 2, 3], chiang ogw: 331/928
b: 5, A: [2, 2, 2, 3], chiang ogw: -25911/59392
b: 6, A: [2, 2, 2, 2, 3], chiang ogw: 83871991/99897344
b: 6, A: [3, 3, 3], chiang ogw: -2994/24389
b: 7, A: [2, 2, 2, 2, 2, 3], chiang ogw: -22125020703/12786860032
b: 7, A: [2, 3, 3, 3], chiang ogw: -36043/780448
b: 8, A: [2, 2, 2, 2, 2, 2, 3], chiang ogw: 133896663195/42006833152
b: 8, A: [2, 2, 3, 3, 3], chiang ogw: -12833319/164089192
b: 9, A: [2, 2, 2, 2, 2, 2, 2, 3], chiang ogw: 9201115703455809/2752959817449472
b: 9, A: [2, 2, 2, 3, 3, 3], chiang ogw: 145655741861/168027332608
b:1, k:0, A:[2]
chiang_a: 1/4
chiang_c: 1/4 

b:1, k:1, A:[]
chiang_a: 3
chiang_c: 3 

b:2, k:0, A:[2, 2]
chiang_a: 29/64
chiang_c: 29/64 

b:2, k:0, A:[3]
chiang_a: 1
chiang_c: 1 

b:2, k:1, A:[2]
chiang_a: -9/16
chiang_c: -9/16 

b:2, k:2, A:[]
chiang_a: -175/116
chiang_c: -175/116 

b:3, k:0, A:[2, 2, 2]
chiang_a: -453/1024
chiang_c: -453

b:9, k:0, A:[2, 2, 2, 2, 2, 2, 2, 2, 2]
chiang_a: -3656653673415738523/352378856633532416
chiang_c: -3656653673415738523/352378856633532416 

b:9, k:0, A:[2, 2, 2, 2, 2, 2, 2, 3]
chiang_a: 9201115703455809/2752959817449472
chiang_c: 9201115703455809/2752959817449472 

b:9, k:0, A:[2, 2, 2, 2, 2, 3, 3]
chiang_a: 43960231444277/21507498573824
chiang_c: 43960231444277/21507498573824 

b:9, k:0, A:[2, 2, 2, 3, 3, 3]
chiang_a: 145655741861/168027332608
chiang_c: 145655741861/168027332608 

b:9, k:0, A:[2, 3, 3, 3, 3]
chiang_a: 535204629/1312713536
chiang_c: 535204629/1312713536 

b:9, k:1, A:[2, 2, 2, 2, 2, 2, 2, 2]
chiang_a: -15323605693326708473241/74087654607200190464
chiang_c: -15323605693326708473241/74087654607200190464 

b:9, k:1, A:[2, 2, 2, 2, 2, 2, 3]
chiang_a: -22702309737836843613/578809801618751488
chiang_c: -22702309737836843613/578809801618751488 

b:9, k:1, A:[2, 2, 2, 2, 3, 3]
chiang_a: -36384176566783961/4521951575146496
chiang_c: -36384176566783961/4521951575146496 

b:9,

In [97]:
chiang_ogw = ChiangOGWInvariant(debug=False)

def experiment(b, k, A, debug=False):
    n = 3
    col = 0
    ool = 0
    cor = 0
    oor = 0 
    
    for t in A.bipart():
        I1 = t[0]
        I2 = t[1]
        bipart_coeff = ClassList.bipart_coeff(A, I1) 

        for i in range(0, n+1):
            for b1 in range(1, b):
                b_ = 0
                if (b - b1) % 4 == 0:
                    b_ = (b - b1) / 4
                else:
                    continue
            
                col += bipart_coeff \
                    * gw_invariant(n, b_, [i, 2, 3] + I1) \
                    * chiang_ogw.chiang_ogw_invariant(n, b1, k, [2, n - i] + I2)
            
        for b1 in range(0, b+1):
            b2 = b - b1
            for k1 in range(0, k+1):
                k2 = k - k1
                ool += bipart_coeff*binomial(k, k1) \
                    * chiang_ogw.chiang_ogw_invariant(n, b1, k1+1, [2] + I1) \
                    * chiang_ogw.chiang_ogw_invariant(n, b2, k2, [2, 3] + I2)
                if debug:
                    print(f"ool, ool_1:{chiang_ogw.chiang_ogw_invariant(n, b1, k1+1, [2] + I1)}\n" +\
                          f"     ool_2: {chiang_ogw.chiang_ogw_invariant(n, b2, k2, [2, 3] + I2)}")

        
        for i in range(0, n+1):
            for b1 in range(1, b):
                b_ = 0
                if (b - b1) % 4 == 0:
                    b_ = (b - b1) / 4
                else:
                    continue
                
                cor += bipart_coeff \
                    * gw_invariant(n, b_, [i, 2, 2] + I1)\
                    * chiang_ogw.chiang_ogw_invariant(n, b1, k, [n-i, 3] + I2)
                
        for b1 in range(0, b+1):
            b2 = b - b1
            for k1 in range(0, k+1):
                k2 = k - k1
                
                oor += bipart_coeff*binomial(k, k1) \
                    * chiang_ogw.chiang_ogw_invariant(n, b1, k1, [2, 2] + I1)\
                    * chiang_ogw.chiang_ogw_invariant(n, b2, k2+1, [3] + I2)
                if debug:
                    print(f"oo3, ool_1:{chiang_ogw.chiang_ogw_invariant(n, b1, k1, [2,2] + I1)}\n" +\
                          f"     ool_2:{chiang_ogw.chiang_ogw_invariant(n, b2, k2+1, [3] + I2)}")

    #print(f"b:{b}, k:{k}, A:{A}, lhs:{col - ool}, rhs:{cor - oor}")
    return (col - ool, cor - oor)



                
        

In [98]:
def get_l1(n, b, k, l2, A1, A2):
    m = n - 3 + 2*b + k + 2*l2 - k*n - 2*l2 * A2 # degree axiom
    if (m % (2*(A1 - 1))) == 0:
        return int(m / (2*(A1 - 1)))
    else:
        -1

In [99]:
lhs, rhs = experiment(6, 0, ClassList.compress([2], 3))

print(f"lhs:{lhs}, rhs:{rhs}")


lhs:61155/59392, rhs:61155/59392


In [100]:
for b in range(1, 10):
    for k in range(0, 7):
        for l2 in range(0, 7):
            for l1 in range(0, 7):
                A1 = 2
                A2 = 3
        
                #l1 = get_l1(3, b, k, l2, A1, A2)
                #if l1 < 0:
                #    continue
                A = ClassList.compress([A1]*l1 + [A2]*l2, 3)
                lhs, rhs = experiment(b, k, A)
                if lhs == 0 and rhs == 0:
                    continue
                print(f"b:{b}, k:{k}, A:{A}\nlhs:{lhs}\nrhs:{rhs} \n")

b:5, k:0, A:[0, 0, 0, 0]
lhs:-19/128
rhs:-19/128 

b:6, k:0, A:[0, 0, 1, 0]
lhs:61155/59392
rhs:61155/59392 

b:6, k:1, A:[0, 0, 0, 0]
lhs:2673/14848
rhs:2673/14848 

b:7, k:0, A:[0, 0, 2, 0]
lhs:-45541939/27557888
rhs:-45541939/27557888 

b:7, k:0, A:[0, 0, 0, 1]
lhs:-2235/7424
rhs:-2235/7424 

b:7, k:1, A:[0, 0, 1, 0]
lhs:-71088373/199794688
rhs:-71088373/199794688 

b:7, k:2, A:[0, 0, 0, 0]
lhs:32930625/49948672
rhs:32930625/49948672 

b:8, k:0, A:[0, 0, 3, 0]
lhs:19794957567/6393430016
rhs:19794957567/6393430016 

b:8, k:0, A:[0, 0, 1, 1]
lhs:15725351/24974336
rhs:15725351/24974336 

b:8, k:1, A:[0, 0, 2, 0]
lhs:156829934513/46352367616
rhs:156829934513/46352367616 

b:8, k:1, A:[0, 0, 0, 1]
lhs:5656799/6243584
rhs:5656799/6243584 

b:8, k:2, A:[0, 0, 1, 0]
lhs:-415226747425/336054665216
rhs:-415226747425/336054665216 

b:8, k:3, A:[0, 0, 0, 0]
lhs:-295804060475/84013666304
rhs:-295804060475/84013666304 

b:9, k:0, A:[0, 0, 4, 0]
lhs:-27607558806739/5933103054848
rhs:-2760755880673

In [103]:
for b in range(0, 10):
        for l2 in range(0, 7):
            for l1 in range(0, 7):
                A1 = 2
                A2 = 3

                A = [A1]*l1 + [A2]*l2
                lhs, rhs = experiment(b, 0, ClassList.compress(A, 3))
                if lhs == 0 and rhs == 0:
                    continue
                print(f"b:{b}, k:{k}, A:{A}\nlhs:{lhs}\nrhs:{rhs} \n")

b:5, k:6, A:[]
lhs:-19/128
rhs:-19/128 

b:6, k:6, A:[2]
lhs:61155/59392
rhs:61155/59392 

b:7, k:6, A:[2, 2]
lhs:-45541939/27557888
rhs:-45541939/27557888 

b:7, k:6, A:[3]
lhs:-2235/7424
rhs:-2235/7424 

b:8, k:6, A:[2, 2, 2]
lhs:19794957567/6393430016
rhs:19794957567/6393430016 

b:8, k:6, A:[2, 3]
lhs:15725351/24974336
rhs:15725351/24974336 

b:9, k:6, A:[2, 2, 2, 2]
lhs:-27607558806739/5933103054848
rhs:-27607558806739/5933103054848 

b:9, k:6, A:[2, 2, 3]
lhs:-29826285365/46352367616
rhs:-29826285365/46352367616 

b:9, k:6, A:[3, 3]
lhs:590337/12487168
rhs:590337/12487168 

